# Model LDA topic

In [1]:
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import pandas as pd
import numpy as np
from gensim.models.ldamodel import LdaModel

#df=pd.read_pickle('./pandas_data_candidatos_premas_lema.pd')
df=pd.read_pickle('./pandas_data_candidatos__premas.pd')
lda_model = LdaModel.load('./lda_models/lda_model_precandi_m48lema') #lda_model_precandi_m48lema
#dictionary= corpora.Dictionary.load('./lda_models/candidatos45.dict')
texts=[]
for a in df.Text:#.Clean
    texts.append(a.split())
    
dictionary = corpora.Dictionary(texts)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [5]:
lda_model.print_topics(num_topics=100 ,num_words=10)

[(0,
  '0.021*"jovem" + 0.015*"deixar" + 0.012*"votar" + 0.012*"pt" + 0.011*"pan" + 0.011*"voltar" + 0.011*"ver" + 0.011*"esquerdo" + 0.010*"bolsonaro" + 0.009*"querer"'),
 (1,
  '0.017*"social" + 0.013*"sentir" + 0.013*"tratar" + 0.011*"semana" + 0.011*"querer" + 0.011*"negro" + 0.010*"experiência" + 0.010*"café" + 0.009*"saudar" + 0.009*"comunidade"'),
 (2,
  '0.035*"lei" + 0.025*"direito" + 0.019*"chamar" + 0.018*"entender" + 0.015*"passar" + 0.015*"deixar" + 0.013*"gostar" + 0.013*"perder" + 0.012*"pensar" + 0.012*"respeitar"'),
 (3,
  '0.023*"mulher" + 0.021*"vidar" + 0.019*"filho" + 0.018*"pai" + 0.017*"casar" + 0.014*"filhar" + 0.013*"história" + 0.012*"gostar" + 0.011*"mãe" + 0.011*"pensar"'),
 (4,
  '0.029*"investimento" + 0.016*"juro" + 0.014*"dívida" + 0.011*"fiscal" + 0.011*"taxar" + 0.011*"cortar" + 0.010*"mercar" + 0.010*"pago" + 0.010*"empresar" + 0.009*"proposto"'),
 (5,
  '0.021*"dólar" + 0.016*"comprar" + 0.014*"economia" + 0.012*"chino" + 0.012*"produzir" + 0.012*"co

In [2]:
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
import pandas as pd
import numpy as np
from gensim.models.ldamodel import LdaModel
#lda_model = LdaModel.load('./lda_models/lda_model_precandi_m40')

# Load word2vec model

In [3]:
import gensim
#doc='/home/kid/Documentos/analisis2018/dynamic-nmf-master-2018/out/w2v-model_setembro3.bin'
doc='/home/kid/Documentos/analisis2018/lda_models/w2v_lema'
model_w2v = gensim.models.Word2Vec.load(doc) 

In [4]:
model_w2v.most_similar('ver')

/home/kid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('vir', 0.7912255525588989),
 ('chegar', 0.7148990631103516),
 ('acontecer', 0.7130476236343384),
 ('tava', 0.6676400899887085),
 ('tô', 0.6584104299545288),
 ('dizer', 0.6460803151130676),
 ('imaginar', 0.6379849910736084),
 ('dar', 0.6273107528686523),
 ('assistir', 0.6267815828323364),
 ('recentemente', 0.6229785680770874)]

# Matrix topic-topic similarity

In [5]:
from gensim import corpora, models

In [6]:
topn_words = {str(i): [word for word, prob in lda_model.show_topic(i, topn=15)] for i in range(0, lda_model.num_topics)}

In [7]:
topn_words['1']

['social',
 'sentir',
 'tratar',
 'semana',
 'querer',
 'negro',
 'experiência',
 'café',
 'saudar',
 'comunidade',
 'pensar',
 'lgbt',
 'passar',
 'sofrer',
 'condição']

In [8]:
vectors_topic=[]
for i in range(len(topn_words)):
    tmp=np.zeros(100)
    count=0
    for word in topn_words[str(i)]:
        try:
            tmp=tmp+model_w2v[word]
            count=count+1
        except:
            print ('no: ',i,word)
            pass
        
    tmp=tmp/count
    vectors_topic.append(tmp)
    
vectors_topic=np.asarray(vectors_topic)

/home/kid/.local/lib/python3.5/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [9]:
vectors_topic.shape

(48, 100)

## Matriz topic-topic

In [10]:
index = gensim.similarities.MatrixSimilarity(gensim.matutils.Dense2Corpus(vectors_topic.T))
similarity_matrix_topic = []
for sims in index:
    similarity_matrix_topic.append(sims)
similarity_matrix_topic = np.array(similarity_matrix_topic)

/usr/local/lib/python3.5/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [11]:
similarity_matrix_topic.shape

(48, 48)

In [12]:
np.savetxt('./lda_models/matrix_simi_precandim48lema2.txt', similarity_matrix_topic)

### Test load matrix

In [56]:
import numpy as np
topic_matrix = np.loadtxt('/home/kid/Documentos/analisis2018/lda_models/matrix_simi_precandim30.txt')

In [57]:
topic_matrix.shape

(30, 30)

## TSS similarity

In [28]:
import math

def Cosine(vec1, vec2) :
    result = InnerProduct(vec1,vec2) / (VectorSize(vec1) * VectorSize(vec2))
    return result

def VectorSize(vec) :
    return math.sqrt(sum(math.pow(v,2) for v in vec))

def InnerProduct(vec1, vec2) :
    return sum(v1*v2 for v1,v2 in zip(vec1,vec2))

def Euclidean(vec1, vec2) :
    return math.sqrt(sum(math.pow((v1-v2),2) for v1,v2 in zip(vec1, vec2)))

def Theta(vec1, vec2) :
    return math.acos(Cosine(vec1,vec2)) + math.radians(10)

def Triangle(vec1, vec2) :
    theta = math.radians(Theta(vec1,vec2))
    return (VectorSize(vec1) * VectorSize(vec2) * math.sin(theta)) / 2

def Magnitude_Difference(vec1, vec2) :
    return abs(VectorSize(vec1) - VectorSize(vec2))

def Sector(vec1, vec2) :
    ED = Euclidean(vec1, vec2)
    MD = Magnitude_Difference(vec1, vec2)
    theta = Theta(vec1, vec2)
    return math.pi * math.pow((ED+MD),2) * theta/360

def TS_SS(vec1, vec2) :
    return Triangle(vec1, vec2) * Sector(vec1, vec2)

vec1 = [1,2]
vec2 = [2,4]

print(Euclidean(vec1,vec2))
print(Cosine(vec1,vec2))
print(TS_SS(vec1,vec2))

2.23606797749979
0.9999999999999998
0.0004639582566999478
